In [1]:
%cd 

/Users/palessandro


In [2]:
%cd Documents/GitHub/scRNAseq-AAE/src

/Users/palessandro/Documents/GitHub/scRNAseq-AAE/src


In [3]:
from keras.utils.vis_utils import model_to_dot

import tensorflow as tf
from tensorflow.keras.utils import plot_model

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import os, sys

from imp import load_source
from models import *
from utils import *

try:
    tf.enable_eager_execution()
except:
    pass

tf.executing_eagerly()
tf.random.set_seed(32)

In [4]:
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras import Model
from tensorflow.keras.losses import mse
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers

In [5]:
parameters = {
    'latent_dim': 10,
    'layers_enc_dim': [200, 100],
    'layers_dec_dim': [100, 200],
    'layers_dis_dim': [200, 100, 50, 40, 30],
    'batch_size': 1000,
    'epochs': 30
}

In [6]:
model = VAE(**parameters)

In [7]:
model.get_parameters()

,Value,Description
batch_size,1000,batch size
epochs,30,number of epochs
alpha,0.1,alpha coeff. in activation function
do_rate,0.1,dropout rate
kernel_initializer,glorot_uniform,kernel initializer of all dense layers
bias_initializer,zeros,bias initializer of all dense layers
l2_weight,None,weight of l2 kernel regularization
l1_weight,None,weight of l1 activity regularization
latent_dim,10,dimension of latent space Z
layer_1_enc_dim,200,dimension of encoder dense layer 1


In [8]:
model.load_data('../data/proc/pbmc3k.h5ad')

Dataset imported.


In [9]:
model.labels

[0,
 2,
 0,
 5,
 4,
 0,
 0,
 3,
 3,
 5,
 2,
 0,
 0,
 1,
 3,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 5,
 0,
 0,
 5,
 3,
 2,
 1,
 3,
 1,
 0,
 1,
 5,
 0,
 4,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 5,
 1,
 5,
 5,
 0,
 0,
 2,
 1,
 0,
 1,
 5,
 0,
 0,
 4,
 5,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 0,
 2,
 0,
 0,
 0,
 1,
 1,
 3,
 1,
 0,
 1,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 1,
 0,
 1,
 1,
 6,
 4,
 1,
 0,
 3,
 4,
 2,
 3,
 2,
 0,
 6,
 1,
 0,
 0,
 0,
 1,
 5,
 0,
 2,
 5,
 0,
 3,
 1,
 2,
 3,
 0,
 0,
 1,
 2,
 4,
 0,
 0,
 0,
 0,
 3,
 2,
 1,
 0,
 0,
 0,
 2,
 0,
 3,
 4,
 2,
 3,
 3,
 2,
 0,
 1,
 4,
 3,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 3,
 1,
 2,
 6,
 2,
 1,
 3,
 2,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 1,
 0,
 6,
 0,
 0,
 2,
 2,
 5,
 2,
 0,
 5,
 1,
 4,
 4,
 0,
 1,
 0,
 2,
 0,
 2,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 1,
 3,
 3,
 0,
 0,
 1,
 5,
 0,
 3,
 4,
 0,
 3,
 3,
 3,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 5,
 0,
 1,
 0,
 0,
 0,
 3,
 5,
 0,
 2,
 1,
 0,
 7,
 1,
 0,
 1,
 0,
 2,


In [10]:
model.rescale_data()

Dataset rescaled.


In [11]:
model.data

array([[-1.7150213e-01, -2.8281140e-01, -4.9584758e-02, ...,
        -1.0294342e-01, -2.0921938e-01, -5.3130406e-01],
       [-2.1462303e-01, -3.7560135e-01, -6.4472154e-02, ...,
        -2.9296505e-01, -3.1336972e-01, -5.9676743e-01],
       [-3.7695917e-01, -2.9723111e-01, -6.9460005e-02, ...,
        -1.7101307e-01, -1.7096411e-01,  1.3792611e+00],
       ...,
       [-2.0712890e-01, -2.5214970e-01, -4.9072269e-02, ...,
        -4.9823590e-02, -1.6114253e-01,  2.0418839e+00],
       [-1.9036458e-01, -2.2776997e-01, -4.4680536e-02, ...,
         1.1567287e-03, -1.3526656e-01, -4.8220244e-01],
       [-3.3385262e-01, -2.5530553e-01, -6.0646087e-02, ...,
        -8.0574334e-02, -1.3037601e-01, -4.7142717e-01]], dtype=float32)

In [12]:
model.layers_enc_dim

[200, 100]

In [13]:
for x,y in enumerate(model.layers_enc_dim):
    print(x+1)
    print(y)

1
200
2
100


In [171]:
encoder=model._build_encoder()

In [172]:
z_mean=encoder.get_layer('z_mean')

In [173]:
model.latent_dim

10

In [174]:
type(z_mean)

tensorflow.python.keras.layers.core.Dense

In [47]:
encoder_input = L.Input(shape=(model.original_dim,), name="X")

x = encoder_input

# add dense layers
for i, nodes in enumerate(model.layers_enc_dim):
    x = L.Dense(nodes,
                name="H_" + str(i + 1),
                kernel_initializer=model.kernel_initializer
                )(x)

    x = L.BatchNormalization(name='BN_' + str(i + 1))(x)

    x = L.LeakyReLU(alpha=model.alpha, name='LR_' + str(i + 1))(x)

    x = L.Dropout(rate=model.do_rate, name='D_' + str(i + 1))(x)

z_mean = L.Dense(model.latent_dim,
                 name='z_mean',
                 kernel_initializer=model.kernel_initializer,
                 bias_initializer=model.bias_initializer)(x)

z_log_var = L.Dense(model.latent_dim,
                    name='z_log_var',
                    kernel_initializer=model.kernel_initializer,
                    bias_initializer=model.bias_initializer)(x)

In [175]:
z_mean

In [135]:
K.shape(z_mean)

<tf.Tensor 'Shape_41:0' shape=(2,) dtype=int32>

In [142]:
K.int_shape(z_mean)[1]

10

In [71]:
K.shape(1)

<tf.Tensor: shape=(0,), dtype=int32, numpy=array([], dtype=int32)>

In [72]:
K.shape([1])

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [99]:
K.shape([[1,2]])

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [104]:
tf.shape(z_mean)[0]

<tf.Tensor 'strided_slice_33:0' shape=() dtype=int32>

In [152]:
K.random_normal(shape=(20,5))

<tf.Tensor: shape=(20, 5), dtype=float32, numpy=
array([[-0.04601388, -1.2070051 ,  1.4035368 ,  0.585008  ,  1.5661281 ],
       [ 0.39359266, -0.15310873, -2.640917  ,  0.0585575 ,  0.47280687],
       [-0.28608295,  0.701203  ,  0.66910434,  0.12501723,  0.48414668],
       [ 1.33069   ,  1.7339227 , -0.98482805,  0.67556363,  0.21576886],
       [-0.7344511 , -0.11199681, -1.2654597 ,  1.5493594 ,  0.509453  ],
       [-1.2817936 , -1.0692198 , -1.2472616 , -0.34810218, -0.2199265 ],
       [-0.05029231, -0.78237134,  1.1249809 , -0.4487792 ,  0.9772685 ],
       [ 0.5371143 , -0.71623814, -0.07685244,  1.2020868 , -1.1730026 ],
       [ 1.6985922 , -0.30303127, -0.50709975, -0.5937182 ,  1.6424066 ],
       [-0.72944236,  1.3355337 ,  0.69631296,  0.34431094, -0.9916968 ],
       [-0.57779926,  0.5668796 ,  1.920081  , -0.01352757,  0.8291644 ],
       [-1.5031109 ,  0.05555005,  1.7687721 , -0.13402446, -0.8920586 ],
       [ 1.5303826 ,  0.6293708 , -0.37285572, -0.53755164,  1.

In [137]:
K.random_normal(shape=(K.shape(z_mean)[0],K.int_shape(z_mean)[1]))

<tf.Tensor 'Add_7:0' shape=(None, 10) dtype=float32>

In [162]:
x = K.variable(np.eye(3, 10)) 

print('shape:', K.shape(x))
print('int_shape:', K.int_shape(x))

shape: tf.Tensor([ 3 10], shape=(2,), dtype=int32)
int_shape: (3, 10)


In [167]:
x = K.variable(np.eye(3, 2)) 

print('shape:', K.shape(x))
print('int_shape:', K.int_shape(x))

shape: tf.Tensor([3 2], shape=(2,), dtype=int32)
int_shape: (3, 2)


In [170]:
K.int_shape(x)[1]

2

In [182]:
encoder

In [6]:
x = tf.convert_to_tensor(np.array([[1,2],[3,4]]).astype('float32'))
y = tf.convert_to_tensor(np.array([[5,6],[7,8]]).astype('float32'))
sampling([x,y])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 13.829188, -38.171467],
       [-38.20907 , -68.279106]], dtype=float32)>

In [7]:
x = tf.convert_to_tensor(np.array([[1,2],[3,4]]).astype('float32'))
y = tf.convert_to_tensor(np.array([[5,6],[7,8]]).astype('float32'))
sampling([x,y])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-1.5013766,  1.4486619],
       [24.68439  , 59.96495  ]], dtype=float32)>

In [22]:
tf.random.set_seed(42)
z_mean = tf.convert_to_tensor(np.array([[1,2],[3,4]]).astype('float32'))
z_log_var = tf.convert_to_tensor(np.array([[5,6],[7,8]]).astype('float32'))
batch = K.shape(z_mean)[0]
dim = K.int_shape(z_mean)[1]
# by default, random_normal has mean = 0 and std = 1.0
epsilon = K.random_normal(shape=(batch, dim),seed=42)
z_mean + K.exp(0.5 * z_log_var) * epsilon

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 17.018486 ,  -1.0975046],
       [ 33.18102  , -39.631786 ]], dtype=float32)>

In [23]:
tf.random.set_seed(42)
z_mean = tf.convert_to_tensor(np.array([[1,2],[3,4]]).astype('float32'))
z_log_var = tf.convert_to_tensor(np.array([[5,6],[7,8]]).astype('float32'))
batch = K.int_shape(z_mean)[0]
dim = K.int_shape(z_mean)[1]
# by default, random_normal has mean = 0 and std = 1.0
epsilon = K.random_normal(shape=(batch, dim),seed=42)
z_mean + K.exp(0.5 * z_log_var) * epsilon

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 17.018486 ,  -1.0975046],
       [ 33.18102  , -39.631786 ]], dtype=float32)>